# PROJEKAT

In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import dlib

In [2]:
def loadImageAnd2Gray(path):
    img_haar = cv2.imread(path)
    return cv2.cvtColor(img_haar, cv2.COLOR_BGR2GRAY)

In [3]:
def display_gray_image(image):
    plt.imshow(image, 'gray')
    
def display_image(image):
    plt.imshow(image)

In [4]:
def draw_coords(image, coords):
    cv2.drawContours(image, [coords], -1, (255,255,255), 3)

In [5]:
def to_np_array(pose_landmarks, el_num=68, dtype="int"):
    coords = np.zeros((el_num, 2), dtype=dtype)
    
    for i in range(0, el_num):
        coords[i] = (pose_landmarks.part(i).x, pose_landmarks.part(i).y)
    
    return coords

In [6]:
def draw_circle(image, coords):
    cv2.circle(image, (coords[0], coords[1]), 25, (255, 255, 255), 2)

# Korisne stvari
cv2.rectangle(img, (x,y), (x+w, y+h), (0, 0 , 255), 3) -> iscrtavanje faca na ulaznoj slici

# Pronalazenje 68 tacaka i cetriranje

In [7]:
predictor_model = "shape_predictor_68_face_landmarks.dat"
face_pose_predictor = dlib.shape_predictor(predictor_model)
DESIRED_LEFT_EYE = (0.35, 0.35)
DESIRED_RIGHT_EYE = (0.65, 0.35)
DESIRED_FACE_WIDTH = 256
DESIRED_FACE_HEIGHT = 256

In [8]:
def align_face(face_rect, verbose=False):

    pose_landmarks = face_pose_predictor(img, face_rect)
    landmark_coords = to_np_array(pose_landmarks)

    left_eye_coords = landmark_coords[36:41]
    right_eye_coords = landmark_coords[42:47]
    
    if verbose:
        draw_coords(img, landmark_coords)
        draw_coords(img, left_eye_coords)
        draw_coords(img, right_eye_coords)

    left_eye_center = left_eye_coords.mean(axis=0).astype("int")
    right_eye_center = right_eye_coords.mean(axis=0).astype("int")

    dx = right_eye_center[1] - left_eye_center[1]
    dy = right_eye_center[0] - left_eye_center[0]
    angle = np.degrees(np.arctan2(dx, dy)) # - 180

    dist = np.sqrt((dx * dx) + (dy * dy))
    desiredDist = (DESIRED_RIGHT_EYE[0] - DESIRED_LEFT_EYE[0])
    desiredDist *= DESIRED_FACE_WIDTH
    scale = desiredDist / dist

    eyes_center = ((left_eye_center[0] + right_eye_center[0]) // 2, (left_eye_center[1] + right_eye_center[1]) // 2)
    M = cv2.getRotationMatrix2D(eyes_center, angle, scale)
    tx = DESIRED_FACE_WIDTH * 0.5
    ty = DESIRED_FACE_HEIGHT * DESIRED_LEFT_EYE[1]
    M[0, 2] += (tx - eyes_center[0])
    M[1, 2] += (ty - eyes_center[1])

    (w, h) = (DESIRED_FACE_WIDTH, DESIRED_FACE_HEIGHT)
    aligned_face = cv2.warpAffine(img, M, (w, h), flags=cv2.INTER_CUBIC)

    return aligned_face

# Ekstrakcija obelezja hog-om

In [9]:
IMAGE_ROWS = 256
IMAGE_COLS = 256
NBINS = 9             # broj binova
CELL_SIZE = (8, 8)    # broj piksela po celiji
BLOCK_SIZE = (3, 3)   # broj celija po bloku

In [10]:
def extract_hog_features(hog, aligned_image):
    return hog.compute(aligned_image)

In [11]:
hog = cv2.HOGDescriptor(_winSize=(IMAGE_COLS // CELL_SIZE[1] * CELL_SIZE[1], 
                                  IMAGE_ROWS // CELL_SIZE[0] * CELL_SIZE[0]),
                        _blockSize=(BLOCK_SIZE[1] * CELL_SIZE[1],
                                    BLOCK_SIZE[0] * CELL_SIZE[0]),
                        _blockStride=(CELL_SIZE[1], CELL_SIZE[0]),
                        _cellSize=(CELL_SIZE[1], CELL_SIZE[0]),
                        _nbins=NBINS)

# Preoznavanje face sa haar-om

In [17]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [61]:
def detect_and_extract_faces_haar(img_path, train=False, verbose=False):
    img = loadImageAnd2Gray(img_path)
    extracted_faces_features = []
    
    faces = face_cascade.detectMultiScale(img, scaleFactor=1.3, minNeighbors=4, minSize=(30, 30), flags=cv2.CASCADE_SCALE_IMAGE)
    
    if train and len(faces) > 1:
        return None
    
    if verbose:
        print("-Faces found: {}".format(len(faces)))

    for (x, y , w ,h) in faces:
        face_rect = dlib.rectangle(x, y, x + w, y + h)
        
        aligned_face = align_face(face_rect)
        
        extracted_face_features = extract_hog_features(hog, aligned_face)
        extracted_faces_features.append(extracted_face_features)
    
    return extracted_faces_features

# Prepoznavanje face sa hog-om

In [19]:
face_detector = dlib.get_frontal_face_detector()

In [62]:
def detect_and_extract_faces_hog(img_path, verbose=False):
    img = loadImageAnd2Gray(img_path)
    extracted_faces_features = []

    faces = face_detector(img, 1)

    if verbose:
            print("-Faces found: {}".format(len(faces)))

    for face_rect in faces:
        aligned_face = align_face(face_rect)

        extracted_face_features = extract_hog_features(hog, aligned_face)
        extracted_faces_features.append(extracted_face_features)

    return extracted_faces_features

In [112]:
path_begin_a = "Andjelina/aj"
path_begin_b = "Bred/bp"
path_end = ".jpeg"

x_train = []
y_train = []

for i in range(10):
    path_a = path_begin_a + str(i + 1) + path_end
    path_b = path_begin_b + str(i + 1) + path_end
    
    hoged = detect_and_extract_faces_haar(path_a, True)
    if hoged != None:
        hoged[0] = hoged[0].flatten()
        x_train.append(hoged[0])
        y_train.append(0)

    hoged = detect_and_extract_faces_haar(path_b, True)
    if hoged != None:
        hoged[0] = hoged[0].flatten()
        x_train.append(hoged[0])
        y_train.append(1)

x_test = load_x()

y_test = [1, 1, 1] 
        
from sklearn.svm import SVC

svm_model_linear = SVC(kernel = 'linear', C = 1).fit(x_train, y_train)
svm_predictions = svm_model_linear.predict(x_test)
accuracy = svm_model_linear.score(x_test, y_test)
print(accuracy)

error: C:\ci\opencv_1512688052760\work\modules\imgproc\src\color.cpp:11048: error: (-215) scn == 3 || scn == 4 in function cv::cvtColor


In [111]:
def load_x(): 
    x_test = []
    
#     path = "Test/aj1_test.jpeg"
#     hoged = detect_and_extract_faces_haar(path)
#     for hog in hoged:
#         hog = hog.flatten()
#         x_test.append(hog)

    
    path = "Test/bp1_test.jpeg"
    hoged = detect_and_extract_faces_haar(path)
    for hog in hoged:
        hog = hog.flatten()
        x_test.append(hog)
    
    path = "Test/bp2_test.jpeg"
    hoged = detect_and_extract_faces_haar(path)
    for hog in hoged:
        hog = hog.flatten()
        x_test.append(hog)
    
    path = "Test/bp3_test.jpeg"
    hoged = detect_and_extract_faces_haar(path)
    for hog in hoged:
        hog = hog.flatten()
        x_test.append(hog)
    
#     print(len(y_test))

    return x_test